In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2023-08-14 17:48:54.903895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 17:48:55.468665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-08-14 17:48:56.244189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 17:48:56.260261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 17:48:56.260428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from datetime import datetime

from hpcom.signal import create_ofdm_parameters, generate_ofdm_signal, decode_ofdm_signal, dbm_to_mw
from hpcom.metrics import get_average_power, get_energy

from hpcom.signal import gen_wdm_bit_sequence
from hpcom.modulation import get_constellation_point, get_modulation_type_from_order, get_scale_coef_constellation
from hpcom.channel import create_channel_parameters

from hpcom.channel import create_channel_parameters, full_line_model_default, full_line_model_wdm, full_line_model, full_line_model_back_to_back, full_line_model_ofdm
from hpcom.signal import create_wdm_parameters, generate_wdm, nonlinear_shift

import tensorflow as tf

import hpcom
import ssfm_gpu

from importlib import reload
# reload(hpcom)

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-14 17:48:57.705703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 17:48:57.705913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 17:48:57.706067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# WDM performance

# OFDM performance

In [5]:
# create parameters
power_range = np.arange(-10, 10, 3)

channel = create_channel_parameters(n_spans=12,
                                    z_span=80,
                                    alpha_db=0.2,
                                    gamma=1.2,
                                    noise_figure_db=4.5,
                                    dispersion_parameter=16.8,
                                    dz=1)

all_results = []

for p_ave_dbm in power_range:
    
    print('p_ave_dbm = ', p_ave_dbm, ' dBm')

    ofdm = hpcom.signal.create_ofdm_parameters(n_carriers=512, 
                                               p_ave_dbm=p_ave_dbm, 
                                               n_symbols=2**11, 
                                               m_order=16,
                                               # symb_freq=34e9,
                                               symb_freq=100e6,
                                               cp_len=0, n_guard=0, n_pilot=0,
                                               n_polarisations=2, seed='fixed')
    
    all_results.append(full_line_model_ofdm(channel, ofdm, verbose=3))

Average signal power (x / y): 0.0031547 / 0.0031541 (has to be close to 0.0031548)
propagation took 1027.1999999999998 ms
search 0 polarisation points took 437.14799999999997 ms
search 0 polarisation BER took 711.708 ms
search 1 polarisation points took 431.088 ms
search 1 polarisation BER took 720.132 ms
BER (x / y): (0.0, 0) / (0.0, 0)
Q^2-factor (x / y): inf / inf
EVM (x / y): 0.24230579412646536 / 0.2421546244125963
MI (x / y): 2.7725833649493055 / 2.772580767323366


{'points': [array([-0.00130944+0.00225042j,  0.0020423 +0.00124363j,
          0.00212662+0.00126127j, ...,  0.0005722 +0.00083679j,
         -0.00238003-0.00115115j, -0.00057252-0.00105486j]),
  array([ 0.00274034-0.00175147j,  0.00194504+0.00142993j,
          0.00036322+0.00240721j, ..., -0.0014589 +0.00190811j,
          0.00080677-0.00070079j, -0.00093145+0.00059773j])],
 'points_orig': [array([-0.00078496+0.00235489j,  0.00235489+0.00078496j,
          0.00235489+0.00078496j, ...,  0.00078496+0.00078496j,
         -0.00235489-0.00078496j, -0.00078496-0.00078496j]),
  array([ 0.00235489-0.00235489j,  0.00235489+0.00078496j,
          0.00078496+0.00235489j, ..., -0.00078496+0.00235489j,
          0.00078496-0.00078496j, -0.00078496+0.00078496j])],
 'points_shifted': [array([-0.00080509+0.0024838j ,  0.00226585+0.00078483j,
          0.00235223+0.00078419j, ...,  0.00073862+0.00069851j,
         -0.00257719-0.00062242j, -0.00078527-0.00091216j]),
  array([ 0.0023136 -0.00229862j,  